In [ ]:
import os
currentdir = os.path.dirname(os.path.abspath(os.getcwd()))

import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 1})

Based on 
- Goldman, Mark S. "Memory without feedback in a neural network." Neuron 61.4 (2009): 621-634.


In [ ]:
def plot_eigenvalues(eigvals):
    plt.figure(figsize=(3, 3))
    plt.scatter(np.real(eigvals), np.imag(eigvals))
    plt.xlim([-1.1,1.1])
    plt.ylim([-1.1,1.1])
    plt.xlabel(r"Re($\lambda$)")
    plt.ylabel(r"Im($\lambda$)");

In [ ]:
tau = .1
dt = .01
N = 100

def simulate_network(W, T, a_weights, x):
    N = W.shape[0]
    r = np.zeros(N) #initial state
    all_r = np.zeros((T,N))
    for t in range(T):
        r += ((-r + r @ W)*dt)/tau + np.multiply(a_weights, x[t])
        all_r[t,:] = r
    return all_r


T = int(10/dt)
times = np.arange(T)*dt

In [ ]:
#input: a_i represents the strength of the external input to unit i
a_weights = np.zeros(N)
a_weights[0] = 1.

#input x(t)
x = np.zeros(T)
x[0] = 1

#connectivity matrix: feedforward network
W = np.diag(np.ones((1, N-1))[0], 1)

In [ ]:
I = 1
all_r = simulate_network(W, T, a_weights, I*x)
output_weights =  np.ones(N)
weighted_sum = np.dot(all_r, output_weights)

In [ ]:
#Figure 1B
#maintained memory of pulse
fig, axs = plt.subplots(2, 1, figsize=(7, 7), sharex=True)
axs[0].plot(times, all_r)
    
axs[1].plot(times, weighted_sum)
axs[1].set_ylim([0,I*1.03])
plt.xlabel("Time (sec)");
plt.ylabel("");

In [ ]:
#Figure 1E: Integration
# step input leads to a linear ramping output with slope proportional to the size of the step
I= .01
x = np.ones(T)
all_r = simulate_network(W, T, a_weights, I*x)
output_weights =  np.ones(N)
weighted_sum = np.dot(all_r, output_weights)

fig, ax = plt.subplots(1, 1, figsize=(7, 4), sharex=True)

ax.plot(times, weighted_sum)
ax.set_ylim([0,10*1.03])
plt.xlabel("Time (sec)");
plt.ylabel("");

### Feedforward Processing of Inputs by a Recurrent Network

In [ ]:
#Feedforward function of feedforward and recurrent networks
#random orthogonal network 
N=3
random_vecs = np.random.randn(N, N)
U, R = np.linalg.qr(random_vecs)

#feedforward connectivity 
T_matrix = np.diag(np.ones((1, N-1))[0], 1)
W = U @ T_matrix @ np.linalg.inv(U)

In [ ]:
T = int(1/dt)
times = np.arange(T)*tau
x = np.zeros(T)
x[0] = 1
a_weights = np.zeros(N)
a_weights[0] = 1.
all_r_ff = simulate_network(T_matrix, T, a_weights, x) 

a_weights = U @ a_weights
all_r_rec =  U.T @ simulate_network(W, T, a_weights, x).T #== np.linalg.inv(U) @ simulate_network(W, T, a_weights, x).T

fig, axs = plt.subplots(2,1,  figsize=(7, 7), sharex=True)
colors = ['red', 'blue', 'blueviolet']
for i in range(N):
    axs[0].plot(times, all_r_ff[:,i], c=colors[i], label=i)
    axs[1].plot(times, all_r_rec.T[:,i], c=colors[i], label=i)
axs[0].set_title("Feedforward architecture")
axs[1].set_title("Recurrent architecture")
plt.xlabel(r"Time (in units of $\tau$)");
axs[0].legend();
axs[1].legend();

### Schur, but Not Eigenvector, Decomposition Reveals Feedforward Interactions between Patterns of Activity

In [ ]:
N = 2
W_pure = np.ones((N,N))/2.

W_fff = np.array([[1,-1],[1,-1]])/2. #functionally feedforward

W_mixed = np.array([[1/2.,-0.3],[1/2.,-0.3]])

weight_list = [W_pure, W_fff, W_mixed]
label_list = ["Pure\nfeedback", "Functionally\nfeedforward", "Mixed"]

a_weights = np.zeros(N)
a_weights[0] = 1.

output_weights =  np.ones(N)

T = int(.5/dt)
times = np.arange(T)*tau
I = 2
x = np.zeros(T)
x[0] = 1

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 2), sharey=True)
for i, W in enumerate(weight_list):
    all_r = simulate_network(W, T, a_weights, I*x)
    
    axs[i].plot(times, all_r[:,0], '-g', times, np.abs(all_r[:,1]), '-c')
    axs[i].set_xlabel(r"Time (in units of $\tau$)")
    axs[i].set_title(label_list[i])
exp_dec = np.exp(-times)
axs[1].plot(times, exp_dec, '--k')
axs[0].set_ylabel("Neural \n response");

In [ ]:
#degenerate eigenvectors: inverse V does not exist
#nonorthogonal eigenvectors: eigenmodes
colormap = np.array(['blue', 'red'])

fig, axs = plt.subplots(2, 3, figsize=(12, 8))
for i, W in enumerate(weight_list):
    eigvals, eigvecs = np.linalg.eig(W)

    if np.all(np.real(eigvecs[:,0]) != np.real(eigvecs[:,1])):
        eigvecs /= np.max(eigvecs, axis=1)
        all_r = np.linalg.inv(eigvecs.T) @ simulate_network(W, T, a_weights, I*x).T 

        axs[0][i].plot(times, all_r[0,:], '-r', times, all_r[1,:], '-b')
        axs[0][i].set_xlabel(r"Time (in units of $\tau$)")
        axs[0][i].set_title(label_list[i])
    else:
        0
        
    for j, (eigval, eigvec) in enumerate(zip(eigvals, eigvecs.T)):
        eignorm = np.linalg.norm(eigval)
        im = axs[1][i].scatter(np.sign(eigvec[0])*eigvec[0], np.sign(eigvec[0])*eigvec[1], c=colormap[j])  #np.where(np.around(eignorm, 10)>0,0,1)
        
    axs[1][i].axhline(0, linestyle='--', color='grey')
    axs[1][i].axvline(0, linestyle='--', color='grey')
    axs[1][i].set_xlim([-1.1,1.1])
    axs[1][i].set_ylim([-1.1,1.1])
    axs[1][i].set_xlabel(r"$x_1$")
    
axs[1][0].set_ylabel(r"$x_2$")
all_r = simulate_network(W, T, a_weights, x)
plt.tight_layout();
axs[0][0].set_ylabel("Neural \n response");

In [ ]:
#schur

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(6, 2), sharey=True)
for i, W in enumerate(weight_list):
    T_mat, U = scipy.linalg.schur(W)
    U /=  np.max(U, axis=1) # np.linalg.norm(U, axis=1)
    U *= np.sign(U[1,:])
    for j, (eigval, eigvec) in enumerate(zip(eigvals, U.T)):
        eignorm = np.linalg.norm(eigval)
        im = axs[i].scatter(eigvec[1], eigvec[0], c=colormap[j])  #np.where(np.around(eignorm, 10)>0,0,1)
        
    axs[i].axhline(0, linestyle='--', color='grey')
    axs[i].axvline(0, linestyle='--', color='grey')
    axs[i].set_xlim([-1.1,1.1])
    axs[i].set_ylim([-1.1,1.1])

    axs[i].set_xlabel(r"$x_1$")
    axs[i].set_title(label_list[i])
    
axs[0].set_ylabel(r"$x_2$")

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 2))
for i, W in enumerate(weight_list):
    T_mat, U = scipy.linalg.schur(W)
    U /= np.sqrt(2)
    U *= np.sign(U[0,:])
    all_r =  U.T @ simulate_network(W, T, a_weights, I*x).T
    
    axs[i].plot(times, all_r[0,:], '-r', times, all_r[1,:], '-b')
    axs[i].set_xlabel(r"Time (in units of $\tau$)")
    axs[i].set_title(label_list[i])
axs[0].set_ylabel("Neural \n response");

#### Some additional networks

In [ ]:
#cyclic network
T = int(10/dt)
times = np.arange(T)*dt

W = np.diag(np.ones((1, N-1))[0], -1) #(feedfoward + one connection last to first unit)
W[0,-1] = 1.

a_weights = np.zeros(N)
a_weights[0] = 1.

I = 1
x = np.zeros(T)
x[0] = 1
all_r = simulate_network(W, T, a_weights, I*x)
output_weights =  np.ones(N)
weighted_sum = np.dot(all_r, output_weights)

In [ ]:
#Figure 
fig, axs = plt.subplots(2, 1, figsize=(7, 7), sharex=True)
axs[0].plot(times, all_r)
axs[1].plot(times, weighted_sum)
axs[1].set_ylim([0,I*1.03])
plt.xlabel("Time (sec)");
plt.ylabel("");

In [ ]:
sns.set_context("notebook", font_scale=1.)
eigvals, eigvecs = np.linalg.eig(W)
plot_eigenvalues(eigvals);

In [ ]:
#line attractor (low-rank network)
N=2
a = np.random.randn(N)
b = np.random.randn(N)
a /= np.linalg.norm(a)
b /= np.linalg.norm(b)
# a, b = (a,b)/np.sqrt(np.inner(a,b))
W = np.outer(a, b)

eigvals, eigvecs = np.linalg.eig(W)
plot_eigenvalues(eigvals)

In [ ]:
x = np.zeros(T)
x[0] = 1
a_weights = np.zeros(N)
a_weights[0] = 1.

all_r = simulate_network(W, T, a_weights, I*x)
output_weights =  np.ones(N)
weighted_sum = np.dot(all_r, output_weights)

In [ ]:
#Figure 
fig, axs = plt.subplots(2, 1, figsize=(7, 7), sharex=True)
axs[0].plot(times, all_r)
axs[1].plot(times, weighted_sum)
axs[1].set_ylim([-1,1.03])
plt.xlabel("Time (sec)");
plt.ylabel("");